# Taller 4  - Competencia: Clasificación de Peptidos Antimicrobianos

Se cuenta con una recopilación de bases de datos de *péptidos* de dos tipos: *antimicrobianos* y *No antimicrobianos*. Para cada conjunto de datos se tienen 1700 descriptores o características fisicoquímicas de los *péptidos*. 
El objetivo de este Notebook es realizar un proceso de clasificación de los péptidos de ambas clases empleando 3 métodos de clasificación: Regresión Logística, Bosque Aleatorio y Descenso de Gradiente Estocástico. Para ello se dividirá la base de datos en un 80% para entrenamiento y 20% para probar los modelos. 

El procedimiento de trabajo es el siguiente:
1. Se realizará el proceso de clasificación con todas las características.
2. Se aplicará un proceso de eliminación de características recursivo para extraer las características más importantes.
3. Se aplicará un método de extracción de características.

En cada paso de medirán diferentes métricas para evaluar el desempeño de los clasificadores.


Los integrates del grupo son:
- Lina Victoria Parra Duque
- Janick Alberto Reales Salas
- Luisa Fernanda Rios Piedrahita



### Clasificación con todas las características


In [1]:
## Se importan algunas librerías requeridas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

Se procede a cargar las bases de datos de péptidos a emplear:

In [2]:
## Base de datos positivos (peptidos antimicrobianos)
positivos = pd.read_csv("DatosPositivos2.csv", index_col = 0)

## Base de datos negativos (peptidos no antimicrobianos)
negativos = pd.read_csv("DatosNegativos2.csv")

## Conjunto de validación
validacion = pd.read_csv("DatosValidacion2.csv", index_col = 0)

print(positivos.shape)
print(negativos.shape)
print(validacion.shape)

(8322, 1760)
(13679, 1760)
(1623, 1761)


In [3]:
## Verificamos las características contenidas en cada base de datos
print(positivos.columns)
print(negativos.columns)
print(validacion.columns)

Index(['sequence', 'length', 'molecular_weight', 'charge', 'charge_density',
       'isoelectric_point', 'gravy', 'instability_index', 'aromaticity',
       'aliphatic_index',
       ...
       'embed_2_91', 'embed_2_92', 'embed_2_93', 'embed_2_94', 'embed_2_95',
       'embed_2_96', 'embed_2_97', 'embed_2_98', 'embed_2_99', 'class'],
      dtype='object', length=1760)
Index(['sequence', 'length', 'molecular_weight', 'charge', 'charge_density',
       'isoelectric_point', 'gravy', 'instability_index', 'aromaticity',
       'aliphatic_index',
       ...
       'embed_2_91', 'embed_2_92', 'embed_2_93', 'embed_2_94', 'embed_2_95',
       'embed_2_96', 'embed_2_97', 'embed_2_98', 'embed_2_99', 'class'],
      dtype='object', length=1760)
Index(['Unnamed: 0.1', 'sequence', 'length', 'molecular_weight', 'charge',
       'charge_density', 'isoelectric_point', 'gravy', 'instability_index',
       'aromaticity',
       ...
       'embed_2_91', 'embed_2_92', 'embed_2_93', 'embed_2_94', 'embed_2_

El conjunto de datos de validación contiene una columba extra (Unnamed: 0.1) con los índices, por lo que se procede a eliminarla para que los tres conjuntos de datos cuenten con el mismo número de columnas. 

In [4]:
## Se elimina columna con índice de datos de validación

validacion = validacion.iloc[:, 1:]
print(validacion.shape)
validacion.head()

(1623, 1760)


,sequence,length,molecular_weight,charge,charge_density,isoelectric_point,gravy,instability_index,aromaticity,aliphatic_index,...,embed_2_91,embed_2_92,embed_2_93,embed_2_94,embed_2_95,embed_2_96,embed_2_97,embed_2_98,embed_2_99,class
0,VVCACRRALCLPRERRAGFCRIRGRIHPLCCRR,33,3897.77,8.691,0.002230,11.404358,-0.112121,84.766667,0.030303,85.757576,...,2.771416,-0.851930,-0.459909,0.909622,1.402783,-3.848056,-0.528822,-0.740751,-0.257650,1
1,GICACRRRFCPNSERFSGYCRVNGARYVRCCSRR,34,4003.64,7.590,0.001896,10.196106,-0.638235,76.035294,0.117647,34.411765,...,2.356963,-0.590644,-0.433246,0.362768,1.204798,-3.838024,-0.859893,-1.086264,0.052278,1
2,LRDLVCYCRSRGCKGRERMNGTCRKGHLLYTLCCR,35,4121.92,6.689,0.001623,9.611023,-0.551429,16.851429,0.057143,64.000000,...,2.762231,-1.182640,-0.908285,0.802487,1.546229,-4.543212,-0.786475,-0.477468,-0.178319,1
3,RRCICTTRTCRFPYRRLGTCIFQNRVYTFCC,31,3838.56,6.589,0.001717,9.802917,-0.200000,53.977419,0.161290,47.096774,...,1.742604,-0.981736,-0.098695,0.614634,0.774174,-2.818301,-0.395545,-0.738405,0.369630,1
4,VCSCRLVFCRRTELRVGNCLIGGVSFTYCCTRV,33,3715.45,3.591,0.000967,8.981384,0.660606,62.042424,0.090909,91.212121,...,1.734715,-0.830263,-0.386610,0.836167,0.859871,-3.665342,-0.922017,-0.364447,-0.055827,1


Lo siguiente a hacer es unir las bases de datos en una sola llamada "*database*", para posteriormente separar la primera columna, la cual contiene los nombres de cada secuencia de péptidos y no es relevate para el proceso de clasificación. 

In [5]:
## se concatenan las bases

database = pd.concat([positivos, negativos, validacion], axis = 0)

In [6]:
# se elimina columna de texto:

sequence = database['sequence']
database = database.iloc[:, 1:]
database.shape

(23624, 1759)

Ahora separamos la columna del target/clase del conjunto de características para aplicar un proceso de *normalización* sobre este último. Con esta normalización queremos llevar todas las características a una misma escala, que permita un mejor desempeño de los modelos.

In [7]:
# Se divide la base de datos

data = database.iloc[:, :-1] # conjunto de carasterísticas
columnas = data.columns      # nombres de las columnas/características
target = database['class']   # clase de los péptidos

In [8]:
## Se aplica normalización sobre el conjunto de características

data_norm = preprocessing.normalize(data, norm='l2', axis=0)


In [9]:
# Se crea un dataframe con el conjunto de características normalizado:

data = pd.DataFrame(data_norm, columns=columnas)
data.head()

,length,molecular_weight,charge,charge_density,isoelectric_point,gravy,instability_index,aromaticity,aliphatic_index,boman_index,...,embed_2_90,embed_2_91,embed_2_92,embed_2_93,embed_2_94,embed_2_95,embed_2_96,embed_2_97,embed_2_98,embed_2_99
0,0.006130,0.005887,0.010263,0.010246,0.007871,-0.009469,0.008451,0.006390,0.000000,0.009508,...,-0.003859,0.004634,0.001525,-0.002609,0.007585,0.003990,-0.005448,-0.000517,-0.008588,0.001712
1,0.005162,0.004815,0.006535,0.007977,0.008063,0.006413,0.002421,0.000000,0.010008,-0.002086,...,-0.001741,0.003078,-0.007457,-0.005970,-0.001380,-0.001512,-0.003999,-0.005291,-0.003582,-0.002984
2,0.005485,0.005955,0.010550,0.010414,0.009411,-0.004293,0.004986,0.009523,0.003307,0.004552,...,-0.001257,0.005854,0.005131,-0.001131,0.013715,0.012943,-0.009549,-0.006825,-0.006487,0.005322
3,0.006453,0.006128,0.004216,0.004044,0.008480,0.003143,0.001968,0.004047,0.008327,0.001710,...,-0.006106,0.008723,-0.000233,-0.006760,0.011987,0.008103,-0.006340,-0.005677,-0.006845,0.006009
4,0.008066,0.007743,0.001172,0.000890,0.006037,0.002832,0.003619,0.003238,0.002220,0.002960,...,-0.013453,0.007337,-0.016623,0.000291,0.008107,0.017724,-0.010720,-0.009756,-0.016503,-0.000963


A continuación, procedemos a dividir el conjunto de datos en datos en un 80% para entrenamiento y 20% para prueba.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)

Ahora cargamos los modelos que vamos a emplear para la clasificación de los datos, además del método de Validación Cruzada, con el cual prodeceremos a medir el desempeño de los clasificadores en el conjunto de entrenamiento.

In [11]:
## se cargan los modelos

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

Entrenamos los tres modelos con los datos de entrenamiento y empleamos validación cruzada, dividiendo el modelo en 5 partes o folds (cv=5). El método de validación cruzada entrena cada modelo con *k-1* folds y lo valida con el fold restante, midiendo la precisión en cada paso. 


In [12]:
## Entrenamos el modelo con Regresión logistica

# Se ajusta el modelo de Regresión Logística (lr) a los datos y se mide el rendimiento promedio
lr = LogisticRegression(C=500) #C=500: 88.3   C=500: 88.0 C=50: 88.3 C=1: 84.6 penalty='l1' demorado
lr.fit(X_train, y_train)

score = cross_val_score(lr, X_train, y_train, cv=5) 
print("Se obtiene un rendimiento de {0:.1%} con Regresión Logística".format(score.mean()))


## Entrenamos el modelo de Bosque Aleatorio y medimos el rendimiento promedio
rf = RandomForestClassifier(random_state=0)  #no afectación al variar criterion, max depth, n_estimator
rf.fit(X_train, y_train)

score = cross_val_score(rf, X_train, y_train, cv=5)
print("Se obtiene un rendimiento de {0:.1%} con Random Forest".format(score.mean()))


## Entrenamos el modelo de Descenso del Gradiente Estocástico y medimos el rendimiento promedio
sgd = linear_model.SGDClassifier(max_iter=1000, tol=1e-3) 
sgd.fit(X_train, y_train)

score = cross_val_score(sgd, X_train, y_train, cv=5)
print("Se obtiene un redimiento de {0:.1%} con SGD".format(score.mean()))

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

Se obtiene un rendimiento de 88.3% con Regresión Logística


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Se obtiene un rendimiento de 87.7% con Random Forest
Se obtiene un redimiento de 85.2% con SGD


Observamos que se obtiene un mayor rendimiento en el conjunto de entrenamiento con el modelo de Regresión Logística, seguido de Random Forest y por último SGD. Sin embargo, la variación entre el rendimiento de los 3 modelos no es muy significativa. 

Utilizamos las métricas precision, recall, F1-score y confusion matrix para medir el desempeño de los clasifidores en el conjunto de prueba. 

La **matriz de confusión** nos compara las predicciones con los datos reales, donde cada columna de la matriz representa el número de predicciones de cada clase, mientras que las filas represetan la clase real. El **precision** nos mide la capacidad del clasificador para no clasificar como positiva una muestra negativa. El **recall** mide la capacidad del clasificador para clasificar las muestras positivas. El **f1-score** es el promedio balaceado entre *precision* y *recall*.


In [13]:
## Evaluación de los clasificadores
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

##Precision
lr_score = precision_score(y_test, lr.predict(X_test), average='weighted')
rf_score = precision_score(y_test, rf.predict(X_test), average='weighted')
sgd_score = precision_score(y_test, sgd.predict(X_test), average='weighted')

print("Se obtiene una precisión de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene una precisión de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene una precisión de {0:.1%} con SGD\n".format(sgd_score))


##Recall
lr_score = recall_score(y_test, lr.predict(X_test), average='weighted')
rf_score = recall_score(y_test, rf.predict(X_test), average='weighted')
sgd_score = recall_score(y_test, sgd.predict(X_test), average='weighted')

print("Se obtiene un recall de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene un recall de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene un recall de {0:.1%} con SGD\n".format(sgd_score))


##F1_score
lr_score = f1_score(y_test, lr.predict(X_test), average='weighted')
rf_score = f1_score(y_test, rf.predict(X_test), average='weighted')
sgd_score = f1_score(y_test, sgd.predict(X_test), average='weighted')

print("Se obtiene un F1-score de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene un F1-score de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene un F1-score de {0:.1%} con SGD".format(sgd_score))


Se obtiene una precisión de 88.8% con Regresión Logística
Se obtiene una precisión de 88.1% con Random Forest
Se obtiene una precisión de 86.4% con SGD

Se obtiene un recall de 88.8% con Regresión Logística
Se obtiene un recall de 87.6% con Random Forest
Se obtiene un recall de 84.4% con SGD

Se obtiene un F1-score de 88.8% con Regresión Logística
Se obtiene un F1-score de 87.2% con Random Forest
Se obtiene un F1-score de 83.4% con SGD


In [14]:
from sklearn.metrics import confusion_matrix

#Confusion matrix
lr_cf = confusion_matrix(y_test, lr.predict(X_test))
print("Matriz de confusión Regresión Lineal:\n", lr_cf, "\n")

rf_cf = confusion_matrix(y_test, rf.predict(X_test))
print("Matriz de confusión Random Forest:\n", rf_cf, "\n")

sgd_cf = confusion_matrix(y_test, sgd.predict(X_test))
print("Matriz de confusión SGD:\n", sgd_cf)

Matriz de confusión Regresión Lineal:
 [[2782  211]
 [ 316 1416]] 

Matriz de confusión Random Forest:
 [[2881  112]
 [ 474 1258]] 

Matriz de confusión SGD:
 [[2943   50]
 [ 685 1047]]


Al igual que en el conjunto de entrenamiento, los modelos de Regresión Logística y Bosque Aleatorio muestran un rendimiento similar respecto a cada métrica, mientras que el modelo SGD muestra un desempeño más bajo a la hora de clasificar las clases de péptidos.

## Aplicando selección de características

A continuación se aplica un método de selección de carasterísticas sobre los datos y luego se miden las métricas para cada modelo. Esto con el objetivo de identificar si extrayendo las características principales significa una mejora en la capacidad de clasificación de los modelos. 

Para ello se utiliza un Eliminador de Características Recursivo, *RFE*, por sus siglas en inglés, al cual se le ingresa el número de características máximo que queremos seleccionar. 


De manera aleatoria se ha decidido probar cada modelo al seleccionar solo 1000, 500 y 100 características. En cada caso se mide el *Accuracy* con objetivo de comparar el desempeño de los modelos.

#### RFE con Regresión Logística

Seleccionando 1000 características

In [15]:
from sklearn.feature_selection import RFE

# se crea modelo con Regresión Logística y se seleccionan 1000 características, eliminando 100 a cada paso
rfe_lr1 = RFE(estimator=lr, n_features_to_select=1000, step = 100, verbose=1)

#Ajustando modelo a datos
rfe_lr1.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_lr1.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1658 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1558 features.


...

Fitting estimator with 1158 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1058 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


89.0% accuracy on test set.


Seleccionando 500 características

In [16]:
# se crea modelo con Regresión Logística y se seleccionan 500 características, eliminando 200 a cada paso. 
rfe_lr2 = RFE(estimator=lr, n_features_to_select=500, step = 200, verbose=1)

#Ajustando modelo a datos
rfe_lr2.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_lr2.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1558 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1358 features.


...

Fitting estimator with 758 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 558 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


88.9% accuracy on test set.


Seleccionando 100 características

In [17]:
# se crea modelo con Regresión Logística y se seleccionan 1000 características
rfe_lr3 = RFE(estimator=lr, n_features_to_select=100, step = 300, verbose=1)

#Ajustando modelo a datos
rfe_lr3.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_lr3.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1458 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 1158 features.


...

Fitting estimator with 558 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 258 features.


C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


87.3% accuracy on test set.


No se presenta un cambio muy significativo en la exactitud del modelo al seleccionar 1000 o 500 características. Mientras que al seleccionar 100 características, la capacidad de clasificación del modelo de Regresión Logística baja.

#### RFE con modelo de Bosque Aleatorio

Seleccionando 1000 características

In [18]:
# se crea modelo con Random Forest y se seleccionan 1000 características, eliminando 100 a cada paso
rfe_rf1 = RFE(estimator=rf, n_features_to_select=1000, step = 100, verbose=1)

#Ajustando modelo a datos
rfe_rf1.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_rf1.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.
Fitting estimator with 1658 features.
Fitting estimator with 1558 features.
Fitting estimator with 1458 features.
Fitting estimator with 1358 features.
Fitting estimator with 1258 features.
Fitting estimator with 1158 features.
Fitting estimator with 1058 features.
88.1% accuracy on test set.


Seleccionando 500 características

In [19]:
# se crea modelo con Random Forest y se seleccionan 500 características, eliminando 200 a cada paso
rfe_rf2 = RFE(estimator=rf, n_features_to_select=500, step = 200, verbose=1)

#Ajustando modelo a datos
rfe_rf2.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_rf2.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.
Fitting estimator with 1558 features.
Fitting estimator with 1358 features.
Fitting estimator with 1158 features.
Fitting estimator with 958 features.
Fitting estimator with 758 features.
Fitting estimator with 558 features.
88.0% accuracy on test set.


Seleccionando 100 características

In [20]:
# se crea modelo con Random Forest y se seleccionan 100 características, eliminando 300 a cada paso
rfe_rf3 = RFE(estimator=rf, n_features_to_select=100, step = 300, verbose=1)

#Ajustando modelo a datos
rfe_rf3.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_rf3.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.
Fitting estimator with 1458 features.
Fitting estimator with 1158 features.
Fitting estimator with 858 features.
Fitting estimator with 558 features.
Fitting estimator with 258 features.
89.2% accuracy on test set.


Se obtuvo una mejor capacidad de clasifición (89.2%) con el modelo al seleccionar el menor número de características. 

#### RFE con Descenso de Gradiente Estocástico

Seleccionando 1000 características

In [21]:
# se crea modelo con SGD y se seleccionan 1000 características, eliminando 100 a cada paso
rfe_sgd1 = RFE(estimator=sgd, n_features_to_select=1000, step = 100, verbose=1)

#Ajustando modelo a datos
rfe_sgd1.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_sgd1.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.
Fitting estimator with 1658 features.
Fitting estimator with 1558 features.
Fitting estimator with 1458 features.
Fitting estimator with 1358 features.
Fitting estimator with 1258 features.
Fitting estimator with 1158 features.
Fitting estimator with 1058 features.
84.8% accuracy on test set.


Seleccionando 500 características

In [22]:
# se crea modelo con SGD y se seleccionan 500 características, eliminando 200 a cada paso
rfe_sgd2 = RFE(estimator=sgd, n_features_to_select=500, step = 200, verbose=1)

#Ajustando modelo a datos
rfe_sgd2.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_sgd2.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.
Fitting estimator with 1558 features.
Fitting estimator with 1358 features.
Fitting estimator with 1158 features.
Fitting estimator with 958 features.
Fitting estimator with 758 features.
Fitting estimator with 558 features.
84.3% accuracy on test set.


Seleccionando 100 características

In [23]:
# se crea modelo con SGD y se seleccionan 100 características, eliminando 300 a cada paso
rfe_sgd3 = RFE(estimator=sgd, n_features_to_select=100, step = 300, verbose=1)

#Ajustando modelo a datos
rfe_sgd3.fit(X_train, y_train)

#calculamos el accuracy

acc = accuracy_score(y_test, rfe_sgd3.predict(X_test))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 1758 features.
Fitting estimator with 1458 features.
Fitting estimator with 1158 features.
Fitting estimator with 858 features.
Fitting estimator with 558 features.
Fitting estimator with 258 features.
82.7% accuracy on test set.


En este caso se alcanzó un *accuracy* de 84.8% al seleccionar 1000 características, y fue dismiuyendo a medida que se seleccionaron menos características para la clasificación. 

Tomamos los casos en que se tuvo un mejor accuracy para medir las métricas de *precision*, *recall*, *f1-score* y *confusion matrix*.

In [25]:
## Evaluación de los clasificadores
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

##Precision
lr_score = precision_score(y_test, rfe_lr1.predict(X_test), average='weighted')
rf_score = precision_score(y_test, rfe_rf3.predict(X_test), average='weighted')
sgd_score = precision_score(y_test, rfe_sgd1.predict(X_test), average='weighted')

print("Se obtiene una precisión de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene una precisión de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene una precisión de {0:.1%} con SGD\n".format(sgd_score))


##Recall
lr_score = recall_score(y_test, rfe_lr1.predict(X_test), average='weighted')
rf_score = recall_score(y_test, rfe_rf3.predict(X_test), average='weighted')
sgd_score = recall_score(y_test, rfe_sgd1.predict(X_test), average='weighted')

print("Se obtiene un recall de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene un recall de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene un recall de {0:.1%} con SGD\n".format(sgd_score))


##F1_score
lr_score = f1_score(y_test, rfe_lr1.predict(X_test), average='weighted')
rf_score = f1_score(y_test, rfe_rf3.predict(X_test), average='weighted')
sgd_score = f1_score(y_test, rfe_sgd1.predict(X_test), average='weighted')

print("Se obtiene un F1-score de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene un F1-score de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene un F1-score de {0:.1%} con SGD".format(sgd_score))

Se obtiene una precisión de 89.0% con Regresión Logística
Se obtiene una precisión de 89.4% con Random Forest
Se obtiene una precisión de 86.6% con SGD

Se obtiene un recall de 89.0% con Regresión Logística
Se obtiene un recall de 89.2% con Random Forest
Se obtiene un recall de 84.8% con SGD

Se obtiene un F1-score de 88.9% con Regresión Logística
Se obtiene un F1-score de 89.0% con Random Forest
Se obtiene un F1-score de 83.9% con SGD


In [26]:
from sklearn.metrics import confusion_matrix

#Confusion matrix
lr_cf = confusion_matrix(y_test, rfe_lr1.predict(X_test))
print("Matriz de confusión Regresión Lineal:\n", lr_cf, "\n")

rf_cf = confusion_matrix(y_test, rfe_rf3.predict(X_test))
print("Matriz de confusión Random Forest:\n", rf_cf, "\n")

sgd_cf = confusion_matrix(y_test, rfe_sgd1.predict(X_test))
print("Matriz de consufión SGD:\n", sgd_cf)

Matriz de confusión Regresión Lineal:
 [[2782  211]
 [ 308 1424]] 

Matriz de confusión Random Forest:
 [[2857  136]
 [ 373 1359]] 

Matriz de consufión SGD:
 [[2938   55]
 [ 661 1071]]


Al aplicar un método de selección de características, donde se tomal 1000 para los modelos de Regresión Logística y SGD, y 100 características para el modelo de Bosque Aleatorio, se obtuvo una ligera mejora en las métricas de los modelos de Regresión Logística y Bosque Aleatorio, mientras que no se observó mejora alguna con el modelo de Descenso de Gradiente Estocástico. 
  

## Aplicando extracción de características

A continuación se aplica un método de selección de carasterísticas sobre los datos y luego se miden las métricas para cada modelo. Como en el caso anterior, se evalúa cada modelo extrayendo 1000, 500 y 100 componentes. 

In [23]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [24]:
# Pipeline con Regresión Lineal

pipe_lr = Pipeline([
        ('reducer', PCA(n_components=1000)),
        ('classifier', LogisticRegression())])

pipe_lr.fit(X_train, y_train)

# Score the accuracy on the test set
accuracy = pipe_lr.score(X_test, y_test)

# Prints the model accuracy
print('{0:.1%} test set accuracy'.format(accuracy))

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


85.2% test set accuracy


In [25]:
# Pipeline con Random Forest

pipe_rf = Pipeline([
        ('reducer', PCA(n_components=1000)),
        ('classifier', RandomForestClassifier())])

pipe_rf.fit(X_train, y_train)

# Score the accuracy on the test set
accuracy = pipe_rf.score(X_test, y_test)

# Prints the model accuracy
print('{0:.1%} test set accuracy'.format(accuracy))

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


76.3% test set accuracy


In [26]:
# Pipeline con SGD

pipe_sgd = Pipeline([
        ('reducer', PCA(n_components=1000)),
        ('classifier', linear_model.SGDClassifier())])

pipe_sgd.fit(X_train, y_train)

# Score the accuracy on the test set
accuracy = pipe_sgd.score(X_test, y_test)

# Prints the model accuracy
print('{0:.1%} test set accuracy'.format(accuracy))

86.3% test set accuracy


In [27]:
## Evaluación de los clasificadores
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

##Precision
lr_score = precision_score(y_test, pipe_lr.predict(X_test), average='weighted')
rf_score = precision_score(y_test, pipe_rf.predict(X_test), average='weighted')
sgd_score = precision_score(y_test, pipe_sgd.predict(X_test), average='weighted')

print("Se obtiene una precisión de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene una precisión de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene una precisión de {0:.1%} con SGD\n".format(sgd_score))


##Recall
lr_score = recall_score(y_test, pipe_lr.predict(X_test), average='weighted')
rf_score = recall_score(y_test, pipe_rf.predict(X_test), average='weighted')
sgd_score = recall_score(y_test,pipe_sgd.predict(X_test), average='weighted')

print("Se obtiene un recall de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene un recall de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene un recall de {0:.1%} con SGD\n".format(sgd_score))


##F1_score
lr_score = f1_score(y_test, pipe_lr.predict(X_test), average='weighted')
rf_score = f1_score(y_test, pipe_rf.predict(X_test), average='weighted')
sgd_score = f1_score(y_test, pipe_sgd.predict(X_test), average='weighted')

print("Se obtiene un F1-score de {0:.1%} con Regresión Logística".format(lr_score))
print("Se obtiene un F1-score de {0:.1%} con Random Forest".format(rf_score))
print("Se obtiene un F1-score de {0:.1%} con SGD".format(sgd_score))

Se obtiene una precisión de 86.3% con Regresión Logística
Se obtiene una precisión de 77.2% con Random Forest
Se obtiene una precisión de 86.7% con SGD

Se obtiene un recall de 85.2% con Regresión Logística
Se obtiene un recall de 76.3% con Random Forest
Se obtiene un recall de 86.3% con SGD

Se obtiene un F1-score de 84.5% con Regresión Logística
Se obtiene un F1-score de 74.4% con Random Forest
Se obtiene un F1-score de 85.9% con SGD


In [28]:
from sklearn.metrics import confusion_matrix

#Confusion matrix
lr_cf = confusion_matrix(y_test, pipe_lr.predict(X_test))
print("Matriz de confusión Regresión Lineal:\n", lr_cf, "\n")

rf_cf = confusion_matrix(y_test, pipe_rf.predict(X_test))
print("Matriz de confusión Random Forest:\n", rf_cf, "\n")

sgd_cf = confusion_matrix(y_test, pipe_sgd.predict(X_test))
print("Matriz de consufión SGD:\n", sgd_cf)

Matriz de confusión Regresión Lineal:
 [[2901   92]
 [ 607 1125]] 

Matriz de confusión Random Forest:
 [[2801  192]
 [ 926  806]] 

Matriz de consufión SGD:
 [[2859  134]
 [ 513 1219]]
